In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import BeamDynamics as bd
import RFTrackTools as rfttools
import copy

In [ ]:
from importlib import reload
reload(bd)

In [ ]:
# %matplotlib inline
# %matplotlib notebook
%matplotlib widget
plt.rcParams['figure.figsize'] = [9.6, 6.4]
defaultColorCycle = plt.rcParams["axes.prop_cycle"].by_key()['color']
# plotFont = {
#     'family' : 'sans-serif',
#     'weight' : 'normal',
#     'size'   : 12
# }
# matplotlib.rc('font', **plotFont)
# plt.rc('legend', fontsize=10)

# Compare Different Solenoid Fieldmaps

In [ ]:
FILE_PATH = '/afs/psi.ch/project/Pcubed/SimulationRuns/fieldmap/custom_solenoid_n_9_cavity.txt'
BZ_CORR_FACTOR = 0.955   # For L_SOLENOID = 0.65
BZ_SOLENOID = 1   # [T]
L_SOLENOID = 0.65   # [m]
R_IN_SOLENOID = 0.100   # [m]
L_SEPARATION = 0.2   # [m]

## Generate with analytical formula

In [ ]:
solAnalyticZaxis, solAnalyticBzOnAxis = bd.generate_solenoid_fieldmap(L_SOLENOID, BZ_SOLENOID, R_IN_SOLENOID)

## Load fieldmap from FEM simulation

In [ ]:
solFemMatrix = np.loadtxt(FILE_PATH)
solFemZaxis = solFemMatrix[:,0]
solFemBzOnAxis = BZ_SOLENOID * BZ_CORR_FACTOR * solFemMatrix[:,1]
# solFemFieldmapStep = solFemMatrix[1,0] - solFemMatrix[0,0]

## Compare

In [ ]:
fix1, ax1 = plt.subplots()
ax1.plot(solAnalyticZaxis, solAnalyticBzOnAxis)
ax1.plot(solFemZaxis, solFemBzOnAxis)
ax1.set_xlabel('z [m]')
ax1.set_ylabel('Bz [T]')
ax1.grid()

In [ ]:
L_SPACING = L_SOLENOID + L_SEPARATION

## Fieldmap of HTS AMD

In [ ]:
FILE_PATH = '/afs/psi.ch/project/Pcubed/SimulationRuns/fieldmap/narrow_no_W.txt'
amd = np.loadtxt(FILE_PATH, delimiter=',')
AMD_DELTA_Z = amd[1,0] - amd[0,0]

In [ ]:
fig3, ax3 = plt.subplots()
_ = ax3.plot(amd[:,0], amd[:,1])
ax3.set_xlabel('z [m]')
ax3.set_ylabel('Bz [T]')
ax3.grid()

## Fieldmap from BiLFINGER, SC Solenoids around RF structures

In [ ]:
FILE_PATH = '/mnt/8414/CHARTProject/Pcubed/Components/Magnets/Experiment/SolenoidAroundRF/Bilfinger/20220519_FieldOnAxis_Sum_ExportedData.txt'
solBilfSum = np.loadtxt(FILE_PATH, delimiter=',')

In [ ]:
L_SOLENOID = 0.17
SEPARATION_SOLENOID = 0.3
BZ_SOLENOID = 3.
R_IN_SOLENOID = 0.22
solAnalyticBilfSingleZ, solAnalyticBilfSingleBzOnAxis = bd.generate_solenoid_fieldmap(L_SOLENOID, BZ_SOLENOID, R_IN_SOLENOID)
# solAnalyticBilfSumBzOnAxis = np.zeros(solAnalyticBilfSingleBzOnAxis.shape)
# for solInd in range(4):
#     solAnalyticBilfSumBzOnAxis += np.interp

In [ ]:
R_IN_COIL = 0.22
R_OUT_COIL = 0.23
L_SOLENOID = 0.17
J = 255e6
Z_CENTERS = [-1.05, -0.76, -0.45, -0.155, 0.155, 0.45, 0.76, 1.05]
DIST_AMD_TO_RF_MIDDLE = 1.535   # [m]
solAnalytic2BilfZ = np.arange(amd[0,0]-DIST_AMD_TO_RF_MIDDLE, 2.0, AMD_DELTA_Z)
solAnalytic2BilfSingleBzOnAxis = []
solAnalytic2BilfSumBzOnAxis = np.zeros(solAnalytic2BilfZ.shape)
for zCenter in Z_CENTERS:
    solAnalytic2BilfSingleBzOnAxis.append(bd.generate_solenoid_fieldmap_wilson(
        solAnalytic2BilfZ, zCenter, R_IN_COIL, R_OUT_COIL, L_SOLENOID/2., J
    ))
    solAnalytic2BilfSumBzOnAxis += solAnalytic2BilfSingleBzOnAxis[-1]

In [ ]:
fig2, ax2 = plt.subplots()
hLeg = []
hPlot, = ax2.plot(solBilfSum[:,0], solBilfSum[:,1], color=defaultColorCycle[0])
# ax2.plot(solAnalyticBilfSingleZ-0.15, solAnalyticBilfSingleBzOnAxis)
# ax2.plot(solAnalyticBilfSingleZ-0.45, solAnalyticBilfSingleBzOnAxis)
# ax2.plot(solAnalyticBilfSingleZ-0.75, solAnalyticBilfSingleBzOnAxis)
# ax2.plot(solAnalyticBilfSingleZ-1.05, solAnalyticBilfSingleBzOnAxis)
hLeg.append(hPlot)
hPlot, = ax2.plot(
    solAnalytic2BilfZ, solAnalytic2BilfSumBzOnAxis, '-', color=defaultColorCycle[1])
hLeg.append(hPlot)
for singleBzOnAxis, zCenter in zip(solAnalytic2BilfSingleBzOnAxis, Z_CENTERS):
    hPlot, = ax2.plot(solAnalytic2BilfZ, singleBzOnAxis, '--', color=defaultColorCycle[1])
hLeg.append(hPlot)
ax2.set_xlabel('z [m]')
ax2.set_ylabel('Bz [T]')
ax2.legend(hLeg, [
    "Field profile from BiLFINGER",
    "Analytic reconstruction (Jaap's formula), Single coils",
    "Analytic reconstruction (Jaap's formula)"
])
ax2.grid()

## Superposition AMD + Solenoids Around RF Structures, P3 Experiment

In [ ]:
# Z_TARGET_REL_TO_PEAK = 0.   # [m]
# Z_RF_MIDDLE_FROM_TARGET = 0.850   # [m]
totBzOnAxis = solAnalytic2BilfSumBzOnAxis.copy()
totBzOnAxis[0:amd.shape[0]] += amd[:,1]

In [ ]:
fig4, ax4 = plt.subplots()
ax4.plot(
    amd[:,0], amd[:,1], '-', color=defaultColorCycle[0],
    label="AMD"
)
ax4.plot(
    solAnalytic2BilfZ+DIST_AMD_TO_RF_MIDDLE, solAnalytic2BilfSumBzOnAxis, '-', color=defaultColorCycle[1],
    label="Analytic reconstruction of BiLFINGER (Jaap's formula)"
)
ax4.plot(
    solAnalytic2BilfZ+DIST_AMD_TO_RF_MIDDLE, totBzOnAxis, '-', color=defaultColorCycle[2],
    label="Total field on axis"
)
ax4.set_xlabel('z [m]')
ax4.set_ylabel('Bz [T]')
ax4.grid()

## Superposition AMD + Solenoids Around RF Structures, FCC-ee Injector

In [ ]:
Z_MAX = 10.   # [m]
zLinac = np.arange(amd[0,0], Z_MAX, AMD_DELTA_Z)
TARGET_EXIT_Z = 0.041   # [m]
BzOnAxisLinac = np.zeros(zLinac.shape)
BzOnAxisLinac[0:amd.shape[0]] += amd[:,1]

In [ ]:
SOL_HOMOG_BZ = 0.5   # [T]
indZConstBzStart = np.nonzero(amd[:,1] > SOL_HOMOG_BZ)[0][-1] + 1

In [ ]:
RF_L_STRUCTURE = 1.2   # [m]
RF_L_FLANGE = 0.05461   # [m]
RF_L_MECH_MARGIN = 0.01   # [m]
RF_CELLS_PER_PERIOD = 3.
RF_L_CELL = bd.C / rfttools.RF_CLIC_FREQ / RF_CELLS_PER_PERIOD
RF_N_STRUCTURES = 5
RF_SEPARATION = 0.2   # [m]
RF_SPACING = RF_L_STRUCTURE + RF_SEPARATION
SOL_R_IN_COIL = 0.130   # [m]
SOL_R_OUT_COIL = 0.250   # [m]
SOL_L = RF_L_STRUCTURE - RF_L_FLANGE - RF_L_CELL - RF_L_MECH_MARGIN
SOL_SPACING = RF_SPACING
SOL_J = 3.54e6   # [A/m2]
AMD_Z_EXIT = 0.2253   # [m]
INITIAL_L = 0.   # [m]
Z_CENTER_FIRST_SOL = AMD_Z_EXIT + INITIAL_L + RF_L_STRUCTURE / 2.
Z_CENTER_SOLS = Z_CENTER_FIRST_SOL + np.arange(RF_N_STRUCTURES) * SOL_SPACING
# DIST_AMD_TO_RF_MIDDLE = 1.535   # [m]
# solAnalytic2BilfZ = np.arange(amd[0,0]-DIST_AMD_TO_RF_MIDDLE, 2.0, AMD_DELTA_Z)
BzOnAxisSingles = []
for zCenter in Z_CENTER_SOLS:
    BzOnAxisSingles.append(bd.generate_solenoid_fieldmap_wilson(
        zLinac, zCenter, SOL_R_IN_COIL, SOL_R_OUT_COIL, SOL_L/2., SOL_J
    ))
    BzOnAxisLinac += BzOnAxisSingles[-1]

In [ ]:
fig5, ax5 = plt.subplots()
hLeg = []
hPlot, ax5.vlines(TARGET_EXIT_Z, np.min(amd[:,1]), np.max(amd[:,1]), linestyles='-', colors=defaultColorCycle[3])
hLeg.append(hPlot)
hPlot, ax5.plot(amd[:,0], amd[:,1], '--', color=defaultColorCycle[0])
hLeg.append(hPlot)
hPlot, ax5.plot([amd[indZConstBzStart,0], zLinac[-1]], [SOL_HOMOG_BZ, SOL_HOMOG_BZ], '--', color=defaultColorCycle[0])
hLeg.append(hPlot)
for BzOnAxis, zCenter in zip(BzOnAxisSingles, Z_CENTER_SOLS):
    hPlot, = ax5.plot(zLinac, BzOnAxis, '--', color=defaultColorCycle[1])
hLeg.append(hPlot)
hPlot, = ax5.plot(zLinac, BzOnAxisLinac, '-', color=defaultColorCycle[2])
hLeg.append(hPlot)
ax5.set_xlabel('z [m]')
ax5.set_ylabel('Bz [T]')
ax5.legend(hLeg, [
    "Target exit",
    "AMD",
    "Homogeneous solenoidal channel",
    "Single solenoids around RF",
    "Total"
])
ax5.grid()

<div class="alert alert-block alert-success">
Some good news.
</div>

<div class="alert alert-block alert-warning">
Some warning.
</div>

<div class="alert alert-block alert-danger">
Some danger.
</div>